In [1]:
import torch
import torch.nn as nn
import random
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from tqdm import tqdm
import pickle 
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym
import numpy as np
import collections 
import cv2
import matplotlib.pyplot as plt
import os

c:\users\admin\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
init_size = 50
pop_size = 50
time_remainning = 400
mutate1_prob = 0.05
mutate2_prob = 0.02
mutate3_prob = 0.05
mutate4_prob = 0.05
crossover_prob = 0.1
status = {"small" : 0, "tall" : 1, "fireball" : 2}

# GA upload


In [3]:
class Individual:
    def __init__(self, actions=None, fitness =None, score = None, dis = None):
        self.fitness = 0
        self.score = 0
        self.dis = 0
        if actions == None:
            actions = []
            for i in range(init_size):
                action = random.randrange(0, len(SIMPLE_MOVEMENT))
                rep = random.randrange(1, 12)
                for j in range(rep):
                    actions.append(action)
            self.actions = actions
            self.evaluate()
        else:
            if fitness == None:
                self.actions = actions
                self.evaluate()
            else:
                self.actions = actions
                self.fitness = fitness
                self.score = score
                self.dis = dis
                
    def evaluate(self):
        fitness = 0
        tmp_actions = []
        cur_status = 0
        reward = 0
        done = 0 
        info = 0
        state = env.reset()
        for action in self.actions:
            for qq in range(stack_frame):
                state, reward, done, info = env.step([action])
                if info[0]['flag_get'] == True:
                    tmp_actions.append(action)
                    fitness += reward[0]
                    self.dis = info[0]['x_pos']
                    self.score = info[0]['score']
                    self.actions = tmp_actions
                    self.fitness = fitness + info[0]['score'] * 5
                    return 
                if  'terminal_observation' in info[0] or status[info[0]['status']] < cur_status:
                    for i in range(24):
                        if len(tmp_actions) > 1:
                            del tmp_actions[-1]
                    self.actions = tmp_actions
                    self.evaluate()
                    return
                cur_status = status[info[0]['status']]
                fitness += reward[0]
            tmp_actions.append(action)
        self.dis = info[0]['x_pos'] 
        self.score = info[0]['score']
        self.actions = tmp_actions
        self.fitness = fitness + info[0]['score'] * 5

In [4]:
class Population:
    def __init__(self, save_file = "", gen = 0):
        self.pop = []
        self.gen = 0
        if(save_file == ""):
            for i in range(0, pop_size):
                print(i)
                self.pop.append(Individual())
        else:
            self.loadFileToPop(save_file, gen)

    def evolve(self):
        for ind in self.pop:
            print(ind.fitness, ind.score, ind.dis)
        print("--------------------------------------------")
        length = len(self.pop)
        for i in range(0, length - pop_size):
            del self.pop[-1]
        a = random.randrange(0, int(pop_size / 10))
        while True:
            b = random.randrange(0, int(pop_size / 5))
            if b != a:
                break
        self.pop[a] = self.mutate2(self.pop[a])
        self.pop[a] = self.mutate3(self.pop[a])
        self.pop[b] = self.mutate2(self.pop[b])
        self.pop[b] = self.mutate3(self.pop[b])
        for i, ind in enumerate(self.pop):
            alpha = random.random()
            if alpha <= mutate1_prob:
                self.pop[i] = self.mutate1(self.pop[i])
            elif alpha <= mutate1_prob + mutate2_prob:
                self.pop[i] = self.mutate2(self.pop[i])
            elif alpha <= mutate1_prob + mutate2_prob + mutate3_prob:
                self.pop[i] = self.mutate3(self.pop[i])
            elif alpha <= mutate1_prob + mutate2_prob + mutate3_prob + mutate4_prob:
                self.pop[i] = self.mutate4(self.pop[i])    
        for i in range(int(crossover_prob * pop_size)):
            a = random.randrange(0, int(pop_size / 5))
            while True:
                b = random.randrange(0, pop_size)
                if b != a:
                    break
            ind1, ind2 = self.crossOver(copy.deepcopy(self.pop[a]), copy.deepcopy(self.pop[b]))
            self.pop.append(ind1)
            self.pop.append(ind2)
        self.pop.sort(key=lambda x: x.fitness, reverse=True)
        self.gen += 1

    def crossOver(self, ind1, ind2):
        child1 = []
        child2 = []
        cut_ind = random.randrange(0, min(len(ind1.actions), len(ind2.actions)))
        for i in range(0, cut_ind):
            child1.append(ind1.actions[i])
            child2.append(ind2.actions[i])
        child1.append(int((ind1.actions[cut_ind] + ind2.actions[cut_ind]) / 2))
        child2.append(int((ind1.actions[cut_ind] + ind2.actions[cut_ind]) / 2))
        for i in range(cut_ind + 1, len(ind2.actions)):
            child1.append(ind2.actions[i])
        for i in range(cut_ind + 1, len(ind1.actions)):
            child2.append(ind1.actions[i])
        return Individual(child1), Individual(child2)

    # chen vao giua chuoi hanh dong
    def mutate1(self, ind):
        t = copy.deepcopy(ind)
        start = random.randrange(0, len(t.actions))
        action = random.randrange(0, len(SIMPLE_MOVEMENT))
        rep = random.randrange(1, 12)
        actions = t.actions[:start] + [action for i in range(rep)] + t.actions[start:]
        x = Individual(actions)
        del t
        if x.fitness > ind.fitness:
            return x
        else:
            return ind

    # chen vao cuoi chuoi hanh dong
    def mutate2(self, ind):
        t = copy.deepcopy(ind)
        res = []
        res2 = []
        rep = random.randrange(1, 12)
        for action in range(1, len(SIMPLE_MOVEMENT) - 1):
            if(action == 2):
                continue
            if(action == 4 and (ind.actions[-1] == 4 or ind.actions[-1] == 2)):
                actions = t.actions + [0] + [action for i in range(rep)]
            else:
                actions = t.actions + [action for i in range(rep)]
            x = Individual(actions)
            print(action, x.fitness)
            if x.fitness > t.fitness:
                res.append(x)
            if x.fitness == t.fitness:
                res2.append(x)
        if len(res) == 0:
            if(len(res2) == 0):
                for i in range(24):
                    if len(t.actions) > 1:
                        del t.actions[-1]
                t.evaluate()
                return t
            else:
                return res2[random.randrange(0, len(res2))]
        return res[random.randrange(0, len(res))]
            
    # chen nhay vao cuoi chuoi hanh dong
    def mutate3(self, ind):
        t = copy.deepcopy(ind)
        rep = random.randrange(4, 12)
        actions = t.actions + [0]
        actions = actions + [4 for i in range(rep)]
        x = Individual(actions)
        del t
        if x.fitness > ind.fitness:
            return x
        else:
            return ind   
            
    # chen chuoi vao giua
    def mutate4(self, ind):
        t = copy.deepcopy(ind)
        rep = random.randrange(1, 12)
        action = random.randrange(0, len(SIMPLE_MOVEMENT))
        start = random.randrange(0, len(t.actions))
        actions = t.actions
        for i in range(start, min(len(actions), start + rep)):
            actions[i] = action
        x = Individual(actions)
        if x.fitness > ind.fitness:
            return x
        else:
            return ind
    
    def savePopToFile(self, save_file):
        save = open(save_file, 'w')
        for ind in self.pop:
            for j in ind.actions:
                save.write(str(j) + ' ')
            save.write(str(ind.fitness) + ' ' + str(ind.score) + ' ' + str(ind.dis) + '\n')
        save.close()
    
    def loadFileToPop(self, save_file, gen):
        self.gen = gen
        self.pop = []
        save = open(save_file, 'r')
        lines = save.readlines()
        for line in lines:
            tmp = tuple(map(float, line.split(" ")[:]))
            actions = []
            for i in range(0, len(tmp) - 3):
                actions.append(int(tmp[i]))
            fitness = tmp[-3]
            score = tmp[-2]
            dis = tmp[-1]
            self.pop.append(Individual(actions, fitness, score, dis))
        save.close()


#Enviroment Wrapper


In [5]:
class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env=None, skip=4):
        super(MaxAndSkipEnv, self).__init__(env)
        self._obs_buffer = collections.deque(maxlen=2)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            self._obs_buffer.append(obs)
            total_reward += reward
            if done:
                break
        max_frame = np.max(np.stack(self._obs_buffer), axis=0)
        return max_frame, total_reward, done, info

    def reset(self):
        self._obs_buffer.clear()
        obs = self.env.reset()
        self._obs_buffer.append(obs)
        return obs


class ProcessFrame84(gym.ObservationWrapper):
    def __init__(self, env=None):
        super(ProcessFrame84, self).__init__(env)
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)

    def observation(self, obs):
        return ProcessFrame84.process(obs)

    @staticmethod
    def process(frame):
        if frame.size == 240 * 256 * 3:
            img = np.reshape(frame, [240, 256, 3]).astype(np.float32)
        else:
            assert False, "Unknown resolution."
        img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
        resized_screen = cv2.resize(img, (84, 110), interpolation=cv2.INTER_AREA)
        x_t = resized_screen[18:102, :]
        x_t = np.reshape(x_t, [84, 84, 1])
        return x_t.astype(np.uint8)


class ImageToPyTorch(gym.ObservationWrapper):
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]),
                                                dtype=np.float32)

    def observation(self, observation):
        return np.moveaxis(observation, 2, 0)


class ScaledFloatFrame(gym.ObservationWrapper):
    def observation(self, obs):
        return np.array(obs).astype(np.float32) / 255.0


class BufferWrapper(gym.ObservationWrapper):
    def __init__(self, env, n_steps, dtype=np.float32):
        super(BufferWrapper, self).__init__(env)
        self.dtype = dtype
        old_space = env.observation_space
        self.observation_space = gym.spaces.Box(old_space.low.repeat(n_steps, axis=0),
                                                old_space.high.repeat(n_steps, axis=0), dtype=dtype)

    def reset(self):
        self.buffer = np.zeros_like(self.observation_space.low, dtype=self.dtype)
        return self.observation(self.env.reset())

    def observation(self, observation):
        self.buffer[:-1] = self.buffer[1:]
        self.buffer[-1] = observation
        return self.buffer


def make_env(env):
    env = MaxAndSkipEnv(env)
    env = ProcessFrame84(env)
    env = ImageToPyTorch(env)
    env = BufferWrapper(env, 4)
    env = ScaledFloatFrame(env)
    return JoypadSpace(env, SIMPLE_MOVEMENT)

#DQN

In [6]:
class DQNSolver(nn.Module):

    def __init__(self, input_shape, n_actions):
        super(DQNSolver, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )

        conv_out_size = self._get_conv_out(input_shape)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )
    
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        return self.fc(conv_out)
    

In [7]:
class DQNAgent:

    def __init__(self, state_space, action_space, max_memory_size, batch_size, gamma, lr,
                 dropout, exploration_max, exploration_min, exploration_decay, double_dq, pretrained):

        self.state_space = state_space
        self.action_space = action_space
        self.double_dq = double_dq
        self.pretrained = pretrained
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        if self.double_dq:  
            self.local_net = DQNSolver(state_space, action_space).to(self.device)
            self.target_net = DQNSolver(state_space, action_space).to(self.device)
            
            if self.pretrained:
                self.local_net.load_state_dict(torch.load(mario_rl + "dq1.pt", map_location=torch.device(self.device)))
                self.target_net.load_state_dict(torch.load(mario_rl + "dq2.pt", map_location=torch.device(self.device)))
                    
            self.optimizer = torch.optim.Adam(self.local_net.parameters(), lr=lr)
            self.copy = 5000  # Copy the local model weights into the target network every 5000 steps
            self.step = 0
        else:  
            self.dqn = DQNSolver(state_space, action_space).to(self.device)
            
            if self.pretrained:
                self.dqn.load_state_dict(torch.load(mario_rl + "dq.pt", map_location=torch.device(self.device)))
            self.optimizer = torch.optim.Adam(self.dqn.parameters(), lr=lr)

        # Create memory
        self.max_memory_size = max_memory_size
        if self.pretrained:
            self.STATE_MEM = torch.load(mario_rl + "STATE_MEM.pt")
            self.ACTION_MEM = torch.load(mario_rl + "ACTION_MEM.pt")
            self.REWARD_MEM = torch.load(mario_rl + "REWARD_MEM.pt")
            self.STATE2_MEM = torch.load(mario_rl + "STATE2_MEM.pt")
            self.DONE_MEM = torch.load(mario_rl + "DONE_MEM.pt")
            with open(mario_rl + "ending_position.pkl", 'rb') as f:
                self.ending_position = pickle.load(f)
            with open(mario_rl + "num_in_queue.pkl", 'rb') as f:
                self.num_in_queue = pickle.load(f)
        else:
            self.STATE_MEM = torch.zeros(max_memory_size, *self.state_space)
            self.ACTION_MEM = torch.zeros(max_memory_size, 1)
            self.REWARD_MEM = torch.zeros(max_memory_size, 1)
            self.STATE2_MEM = torch.zeros(max_memory_size, *self.state_space)
            self.DONE_MEM = torch.zeros(max_memory_size, 1)
            self.ending_position = 0
            self.num_in_queue = 0
        
        self.memory_sample_size = batch_size
        
        # Learning parameters
        self.gamma = gamma
        self.l1 = nn.SmoothL1Loss().to(self.device) # Also known as Huber loss
        self.exploration_max = exploration_max
        self.exploration_rate = exploration_max
        self.exploration_min = exploration_min
        self.exploration_decay = exploration_decay

    def remember(self, state, action, reward, state2, done):
        self.STATE_MEM[self.ending_position] = state.float()
        self.ACTION_MEM[self.ending_position] = action.float()
        self.REWARD_MEM[self.ending_position] = reward.float()
        self.STATE2_MEM[self.ending_position] = state2.float()
        self.DONE_MEM[self.ending_position] = done.float()
        self.ending_position = (self.ending_position + 1) % self.max_memory_size  # FIFO tensor
        self.num_in_queue = min(self.num_in_queue + 1, self.max_memory_size)
        
    def recall(self):
        # Lấy ngẫu nhiên một mẫu để học
        idx = random.choices(range(self.num_in_queue), k=self.memory_sample_size)
        
        STATE = self.STATE_MEM[idx]
        ACTION = self.ACTION_MEM[idx]
        REWARD = self.REWARD_MEM[idx]
        STATE2 = self.STATE2_MEM[idx]
        DONE = self.DONE_MEM[idx]
        
        return STATE, ACTION, REWARD, STATE2, DONE

    def act(self, state):
        # Epsilon-greedy action
        
        if self.double_dq:
            self.step += 1
        if random.random() < self.exploration_rate:  
            return torch.tensor([[random.randrange(self.action_space)]])
        if self.double_dq:
            # Chọn hành động tiếp theo
            return torch.argmax(self.local_net(state.to(self.device))).unsqueeze(0).unsqueeze(0).cpu()
        else:
            return torch.argmax(self.dqn(state.to(self.device))).unsqueeze(0).unsqueeze(0).cpu()

    def copy_model(self):
        
        # Copy tham số từ primary sang target network
        
        self.target_net.load_state_dict(self.local_net.state_dict())
    
    def experience_replay(self):
        
        if self.double_dq and self.step % self.copy == 0:
            self.copy_model()

        if self.memory_sample_size > self.num_in_queue:
            return

        STATE, ACTION, REWARD, STATE2, DONE = self.recall()
        STATE = STATE.to(self.device)
        ACTION = ACTION.to(self.device)
        REWARD = REWARD.to(self.device)
        STATE2 = STATE2.to(self.device)
        DONE = DONE.to(self.device)
        
        self.optimizer.zero_grad()
        if self.double_dq:
            # Double Q-Learning target is Q*(S, A) <- r + γ max_a Q_target(S', a)
            target = REWARD + torch.mul((self.gamma * 
                                        self.target_net(STATE2).max(1).values.unsqueeze(1)), 
                                        1 - DONE)

            current = self.local_net(STATE).gather(1, ACTION.long()) # Local net approximation of Q-value
        else:
            # Q-Learning target is Q*(S, A) <- r + γ max_a Q(S', a) 
            target = REWARD + torch.mul((self.gamma * 
                                        self.dqn(STATE2).max(1).values.unsqueeze(1)), 
                                        1 - DONE)
                
            current = self.dqn(STATE).gather(1, ACTION.long())
        
        loss = self.l1(current, target)
        loss.backward() # Compute gradients
        self.optimizer.step() # Backpropagate error

        self.exploration_rate *= self.exploration_decay

        self.exploration_rate = max(self.exploration_rate, self.exploration_min)

#Runner


In [8]:
def train(mario_map, pretrained, num_episodes, ga_info = None):
    env = gym_super_mario_bros.make(mario_map)
    env = make_env(env) 
    observation_space = env.observation_space.shape
    action_space = env.action_space.n
    agent = DQNAgent(state_space=observation_space,
                     action_space=action_space,
                     max_memory_size=30000,
                     batch_size=32,
                     gamma=0.90,
                     lr=0.00025,
                     dropout=0.,
                     exploration_max=1.0,
                     exploration_min=0.02,
                     exploration_decay=0.99,
                     double_dq=True,
                     pretrained=pretrained)
    if ga_info != None:
        id_ind = ga_info[0]
        id_pop = ga_info[1]
        ga_directory = ga_info[2]
        listGaFile = os.listdir(ga_directory)
        pops = []
        for file in listGaFile:
            g = ""
            for ch in file:
                if ch != '.':
                    g = g + ch
                else:
                    break
            g = int(g)
            try:
                pops.append(Population(ga_directory + '/' + file, g))
            except:
                pass
        pops.sort(key=lambda x: x.gen)
    env.reset()
    total_rewards = []
    total_scores = []
    total_diss = []
    for ep_num in tqdm(range(num_episodes)):
        state = env.reset()
        state = torch.Tensor([state])
        total_reward = 0
        total_score = 0
        total_dis = 0
        steps = 0
        current_score = 0
        current_stt = 0
        idd = 0
        if ga_info != None and random.random() < 0.25 and id_pop < len(pops):
            print(id_pop, id_ind)
            if id_ind == 0:
                id_ind = int((pop_size - 1) / 2)
                id_pop += 1
            else:
                id_ind -= 1
            k  = True
        else:
            k = False
        while True:
            if k:
                if (idd != len(pops[id_pop].pop[id_ind].actions) ):
                    action = torch.tensor([pops[id_pop].pop[id_ind].actions[idd]])
                    idd += 1
                else:
                    break
            else:
                action = agent.act(state)
            steps += 1
            state_next, reward, terminal, info = env.step(int(action[0]))
            reward = reward + (info['score'] - current_score) * 5
            current_stt = status[info['status']]
            current_score = info['score']
            total_reward += reward
            total_score = info['score']
            total_dis = info['x_pos']
            state_next = torch.Tensor([state_next])
            reward = torch.tensor([reward]).unsqueeze(0)
            terminal = torch.tensor([int(terminal)]).unsqueeze(0)
            agent.remember(state, action, reward, state_next, terminal)
            agent.experience_replay()
            state = state_next
            if  terminal:
                break
        
        total_rewards.append(total_reward)
        total_scores.append(total_score)
        total_diss.append(total_dis)

        print(k, "After episode {} is {}".format(ep_num + 1, total_rewards[-1]), total_score, total_dis)
        num_episodes += 1    

    with open(mario_rl + "ending_position.pkl", "wb") as f:
        pickle.dump(agent.ending_position, f)
    with open(mario_rl + "num_in_queue.pkl", "wb") as f:
        pickle.dump(agent.num_in_queue, f)
    with open(mario_rl + "total_rewards.pkl", "wb") as f:
        pickle.dump(total_rewards, f)
    if agent.double_dq:
        torch.save(agent.local_net.state_dict(), mario_rl + "dq1.pt")
        torch.save(agent.target_net.state_dict(), mario_rl + "dq2.pt")
    else:
        torch.save(agent.dqn.state_dict(), mario_rl + "dq.pt")  
    torch.save(agent.STATE_MEM, mario_rl +  "STATE_MEM.pt")
    torch.save(agent.ACTION_MEM, mario_rl + "ACTION_MEM.pt")
    torch.save(agent.REWARD_MEM, mario_rl + "REWARD_MEM.pt")
    torch.save(agent.STATE2_MEM, mario_rl + "STATE2_MEM.pt")
    torch.save(agent.DONE_MEM,  mario_rl + "DONE_MEM.pt")
    if pretrained:
        file_reward = open(mario_rl + "reward_episode.txt", 'a')
        file_score = open(mario_rl + "score_episode.txt", 'a')
        file_dis = open(mario_rl + "dis_episode.txt", 'a')
    else:
        file_reward = open(mario_rl + "reward_episode.txt", 'w')
        file_score = open(mario_rl + "score_episode.txt", 'w')
        file_dis = open(mario_rl + "dis_episode.txt", 'w')
    for i in range(len(total_rewards)):
        file_reward.write(str(total_rewards[i]) + ' ')
        file_score.write(str(total_scores[i]) + ' ')
        file_dis.write(str(total_diss[i]) + ' ')
    file_reward.close()
    file_score.close()
    file_dis.close()
    env.close()
    
    if num_episodes > 500:
        plt.title("Episodes trained vs. Average Rewards (per 500 eps)")
        plt.plot([0 for _ in range(500)] + 
                 np.convolve(total_rewards, np.ones((500,))/500, mode="valid").tolist())
        plt.show()

In [10]:
mario_rl = 'RL_save_model/' # thư mục lưu model RL

# thông số cho GA_data base

id_ind = 24 # cá thể bắt đầu dạy RL
id_pop = 75 # quần thể bắt đầu
name_map = '1-1'
mario_map = 'SuperMarioBros-' + name_map + '-v0' #Chọn map train
ga_database = 'GA_database/mario_ga' + name_map #chọn chỗ database GA

ga_info = [id_ind, id_pop, ga_database]
random.seed(0)
train(mario_map=mario_map, pretrained=True, num_episodes=10, ga_info=ga_info)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]c:\users\admin\appdata\local\programs\python\python39\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256
 10%|████████▎                                                                          | 1/10 [00:08<01:20,  8.93s/it]

False After episode 1 is 3093.0 500 674


 20%|████████████████▌                                                                  | 2/10 [00:12<00:45,  5.65s/it]

False After episode 2 is 230.0 0 295


 30%|████████████████████████▉                                                          | 3/10 [01:50<05:34, 47.81s/it]

False After episode 3 is 4139.0 600 1414


 30%|████████████████████████▉                                                          | 3/10 [01:59<04:39, 39.99s/it]


KeyboardInterrupt: 